## Метод опорных векторов для анализа текстов

Метод опорных векторов (Support Vector Machine, SVM) — один из видов линейных классификаторов. Функционал, который он оптимизирует, направлен на максимизацию ширины разделяющей полосы между классами. Эта ширина тесно связана с обобщающей способностью алгоритма, а ее максимизация позволяет бороться с переобучением.

Одна из причин популярности линейных методов заключается в том, что они хорошо работают на разреженных данных. Так называются выборки с большим количеством признаков, где на каждом объекте большинство признаков равны нулю. Разреженные данные возникают, например, при работе с текстами. Дело в том, что текст удобно кодировать с помощью "мешка слов" — формируется столько признаков, сколько всего уникальных слов встречается в текстах, и значение каждого признака равно числу вхождений в документ соответствующего слова. Ясно, что общее число различных слов в наборе текстов может достигать десятков тысяч, и при это лишь небольшая их часть будет встречаться в одном конкретном тексте.

Можно кодировать тексты хитрее, и записывать не количество вхождений слова в текст, а TF-IDF. Это показатель, который равен произведению двух чисел: TF (term frequency) и IDF (inverse document frequency). Первая равна отношению числа вхождений слова в документ к общей длине документа. Вторая величина зависит от того, в скольки документах выборки встречается это слово. Чем больше таких документов, тем меньше IDF. Таким образом, TF-IDF будет иметь высокое значение для тех слов, которые много раз встречаются в данном документе, и редко встречаются в остальных.

Будем использовать один из датасетов, доступных в scikit-learn'е — 20 newsgroups.

In [18]:
from sklearn import datasets
newsgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])

После выполнения этого кода массив с текстами будет находиться в поле newsgroups.data, номер класса — в поле newsgroups.target.

Одна из сложностей работы с текстовыми данными состоит в том, что для них нужно построить числовое представление. Одним из способов нахождения такого представления является вычисление TF-IDF. В Scikit-Learn это реализовано в классе sklearn.feature_extraction.text.TfidfVectorizer. Преобразование обучающей выборки нужно делать с помощью функции fit_transform, тестовой — с помощью transform.

Реализация SVM-классификатора находится в классе sklearn.svm.SVC. Веса каждого признака у обученного классификатора хранятся в поле coef_. Чтобы понять, какому слову соответствует i-й признак, можно воспользоваться методом get_feature_names() у TfidfVectorizer:

In [24]:
#feature_mapping = vectorizer.get_feature_names()
#print (feature_mapping[i])

Подбор параметров удобно делать с помощью класса sklearn.grid_search.GridSearchCV (При использовании библиотеки scikit-learn версии 18.0.1 sklearn.model_selection.GridSearchCV). Пример использования:

grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = svm.SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X, y)

Первым аргументом в GridSearchCV передается классификатор, для которого будут подбираться значения параметров, вторым — словарь (dict), задающий сетку параметров для перебора. После того, как перебор окончен, можно проанализировать значения качества для всех значений параметров и выбрать наилучший вариант:

for a in gs.grid_scores_:
    #a.mean_validation_score — оценка качества по кросс-валидации
    #a.parameters — значения параметров

## Задание (5 баллов)

__1.__ Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

In [46]:
newsgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])

In [59]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import roc_auc_score

In [48]:
X = newsgroups.data
y = newsgroups.target

 __2.__ Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

In [49]:
vectorizer = TfidfVectorizer(min_df=1)

In [50]:
vict_train = vectorizer.fit_transform(X)
vict_test = vectorizer.transform(X)

In [51]:
idf = vectorizer.idf_
words=vectorizer.get_feature_names()

__3.__ Подберите минимальный лучший параметр C из множества $[10^{-5}, 10^{-4}, ... 10^4, 10^5]$ для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

In [52]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241) 

clf = svm.SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv) 
gs.fit(vict_train, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [53]:
gs.best_estimator_, gs.best_score_

(SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
   max_iter=-1, probability=False, random_state=241, shrinking=True,
   tol=0.001, verbose=False), 0.99328107502799556)

In [54]:
bestC=gs.best_estimator_.C
bestC

1.0

__4.__ Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

In [56]:
clf = svm.SVC(C=bestC,kernel='linear', random_state=241)
clf_result = clf.fit(vict_train, y)

__5.__ Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC).

In [57]:
weights = []
for element in clf_result.coef_.T:
    weights.append(abs(element[0]))

In [65]:
#weights

In [60]:
combine = dict(zip(words,weights))

bestWords = pd.DataFrame(data=combine,index=[0]).transpose()
bestWords.columns = ['weights']
bestWords.sort_values(['weights'], ascending=[False],inplace=True)

In [67]:
bestWords.head()

,weights
space,"(0, 0)\t2.66316478848"
god,"(0, 0)\t1.92037940023"
atheism,"(0, 0)\t1.25468995124"
atheists,"(0, 0)\t1.24918000738"
moon,"(0, 0)\t1.20161118175"


In [62]:
topTenWordsCollection = bestWords.head(10).index.values

newWords = pd.DataFrame(data=topTenWordsCollection)
newWords.columns = ['words']
newWords.words.astype(str)
newWords = newWords.apply(lambda x: x[0].lower(),axis =1)
newWords.sort_values(0, ascending=[True],inplace=True)

In [68]:
newWords

2     atheism
3    atheists
7       bible
1         god
8       keith
4        moon
6    religion
9         sci
5         sky
0       space
dtype: object

In [63]:
collection = newWords.values
print (' '.join(map(lambda x: x, collection)))

atheism atheists bible god keith moon religion sci sky space
